# Hindi Character **Classification**

#### Checking for GPU (optional)

In [1]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Tue Oct  5 03:56:30 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.74       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8    26W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

#### Mounting the Drive

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s \/content/gdrive/My\ Drive/ /mydrive
# !ls /mydrive

Mounted at /content/gdrive


In [3]:
%cd /mydrive/gpuhack
%ls

/content/gdrive/My Drive/gpuhack
dataset/  model.h5  model.json  modelTL.h5  test_2_0/  vggtl.ipynb


#### Importing Packages

In [4]:
import os
from PIL import Image, ImageOps, ImageEnhance, ImageFilter
import numpy as np
from tqdm import tqdm
import matplotlib
from matplotlib import pyplot as plt
import copy
import cv2
from sklearn.model_selection import train_test_split

import tensorflow as tf


from keras.utils.np_utils import to_categorical


Finds the image 
1.   background with Letter
2.   background without Letter

    **And Subracts one from Another**

    And DATA  Augumentation of Image
         

In [5]:
def findImageLetter(path, additions0, additions1):
    
    data =r'dataset/training/training'
    folder_name = 'hi'
    image_ne = []
    
    for filename in tqdm(os.listdir(data + '/' + folder_name),disable=True):
        
        image_name = path.split('_')
        image_name = image_name[0] + '_' + image_name[1]
        
        if additions0[0] in filename and image_name in filename and additions1[0] in filename:
            file_name = data + '/' + folder_name + '/' + filename
            image_n = copy.deepcopy(np.array(Image.open(file_name),dtype = np.uint8))
            image_n = np.asarray(image_n)
            image_ne.append(image_n)
            
    return image_ne


#print(findImageLetter('displayPanel_0',['flip'], ['sharpen']))


def findImageLetterB(path, additions0, additions1):
    
    data = r'dataset/training/training'
    folder_name = 'background'
    image_ne = []
    
    for filename in tqdm(os.listdir(data + '/' + folder_name),disable=True):
        
        image_name = path.split('_')
        image_name = image_name[0] + '_' + image_name[1]
        
        if additions0[0] in filename and image_name in filename and additions1[0] in filename:
            file_name = data + '/' + folder_name + '/' + filename
            image_n = copy.deepcopy(np.array(Image.open(file_name),dtype=np.uint8))
            image_n = np.asarray(image_n)
            image_ne.append(image_n)
            
    return image_ne

def findImageNotLetter(path, additions0 = ['none','flip','rotate'] , additions1 = ['none.jpg','edgeEnhance.jpg','smooth.jpg','sharpen.jpg','blur.jpg']):
    
    image_ne = []
    data = r'dataset/training/training'
    folder_name = 'background'

    for filename in tqdm(os.listdir(data + '/' + folder_name),disable=True):
        
        image_name = path.split('_')
        image_name = image_name[0] + '_' + image_name[1]

        back_image_name = filename.split('_')

        additions3 = back_image_name[2]
        additions4 = back_image_name[3]
        
        for add0 in additions0 :

            for add1 in additions1 :

                if add0 == additions3 and image_name in filename and add1 == additions4:
                    
                    image = copy.deepcopy(Image.open(data + '/' + folder_name + '/' + filename))

                    if add0 in ['flip'] :
                        image = ImageOps.mirror(image)

                    elif add0 in ['rotate'] :
                        image = image.rotate(180)
                    
                    image = np.asarray(image)

                    if(len(image.shape) == 4):
                        image = image[0,:,:,:]

                    image_ne.append(image)
                    return image_ne
        
    return image_ne

def subtractImages(image0, image1):
    image_1 = np.asarray(image0)
    image_2 = np.asarray(image1)

    black_pixels = np.where(
        (image_2[:, :, 0] < 4) &
        (image_2[:, :, 1] < 4) &
        (image_2[:, :, 2] < 4)
    )

    # set those pixels to white
    # image_2[black_pixels] = [255, 255, 255]

    image_final = cv2.absdiff(image_1, image_2)
    kernel = np.ones((2, 2), np.uint8)

    opening = cv2.morphologyEx(image_final, cv2.MORPH_OPEN, kernel)

    image_final = cv2.cvtColor(opening, cv2.COLOR_BGR2GRAY)
    ret, image_final = cv2.threshold(image_final, 17, 255, cv2.THRESH_BINARY)
    final = cv2.subtract(255, image_final)

    NewImg = []
    for i in final:
        NewImg.append([[x, x, x] for x in i])

    return NewImg

def fileExists(data,folder_name,additions0,additions1,file_name):

    exists = 0
    n1 = 0
    n2 = 0

    if(folder_name == 'background'):
        n1 = 2
        n2 = 3
    else:
        n1 = 4
        n2 = 5

    for filename in tqdm(os.listdir(data + '/' + folder_name),disable=True):
        
        image_name = filename.split('_')
        #fin_image_name = image_name[0] + '_' + image_name[1]
        
        if additions0[0] == image_name[n1] and additions1[0] == image_name[n2] and file_name in filename:
            exists = 1

    return exists

import random
def random_att(x0,x1,img):

    fin_att = []
    a0 = ['flip','rotate']
    a1 = ['blur','sharpen','edgeEnhance','smooth']

    r0 = random.sample(a0, 1)
    r1 = random.sample(a1, 1)

    if x0 in r0 :
        a0.remove(r0)
        r0 = random.sample(a0, 1)
    
    if x1 in r1:
        a1.remove(r1)
        r1 = random.sample(a1, 1)
    
    if 'blur' in r1 :
        img = img.filter(ImageFilter.BLUR)

    elif 'sharpen' in r1 :
        img = img.filter(ImageFilter.SHARPEN)
        
    elif 'edgeEnhance' in r1 :
        img = img.filter(ImageFilter.EDGE_ENHANCE)

    elif 'smooth' in r1 :
        img = img.filter(ImageFilter.SMOOTH)

    if 'flip' in r0 :
        img = ImageOps.mirror(img)

    elif 'rotate' in r0 :
        img = img.rotate(180)
    
    return r0,r1,img



Loading two types of images 

1.   Subracted (1)
2.   Background without letter (0)



In [8]:
def load_subdata(data):

    X1 = []
    y1 = []

    #X2 = []
    #y2 = []

    for folder_name in os.listdir(data):

        c_i = 0;

        if not folder_name.startswith('.'):
            
            if folder_name in ['background'] :
                label = 0;
                n1 = 2;
                n2 = 3;
            elif folder_name in ['hi'] :
                label = 1;
                n1 = 4
                n2 = 5

        count_imag = 0;

        for filename in tqdm(os.listdir(data + '/' + folder_name)):

            image = copy.deepcopy(np.array(Image.open(data +'/'+ folder_name + '/' + filename),dtype=np.uint8))
            
            image_name = filename.split('_')
            fin_image_name = image_name[0] + '_' + image_name[1]

            additions0 = []
            additions1 = []

            additions0.append(image_name[n1])
            additions1.append(image_name[n2])

            if folder_name in ['hi'] :

                exist_images = findImageLetterB(fin_image_name,additions0,additions1)

                if exist_images :
            
                    image_letter = subtractImages(exist_images[0],image)
                    #image_letter = cv2.resize(image_letter, (128, 128))
                    X1.append(image_letter)
                    y1.append(1)

                    for i in range(1,3):
                        image = copy.deepcopy(Image.open(data +'/'+ folder_name + '/' + filename))
                        r0,r1,new_img = random_att(additions0,additions1,image)
                        exist_images = findImageLetterB(fin_image_name,r0,r1)

                        if exist_images:
                            image_letter = subtractImages(exist_images[0],new_img)
                            #image_letter = cv2.resize(image_letter, (128, 128))
                            X1.append(image_letter)
                            y1.append(1)
                        
                        else :
                            non_exist_images = findImageNotLetter(fin_image_name)
                    
                            #print(filename)
                            non_exist_images = np.asarray(non_exist_images)
                            
                            if(len(non_exist_images.shape)) == 4:
                                non_exist_images = non_exist_images[0,:, :, :]
                                
                            image_letter = subtractImages(non_exist_images,new_img)
                            #image_letter = cv2.resize(image_letter, (128, 128))
                            X1.append(image_letter)
                            y1.append(1)

                    c_i += 1 

                else :

                    non_exist_images = findImageNotLetter(fin_image_name)
                    
                    #print(filename)
                    non_exist_images = np.asarray(non_exist_images)
                    
                    if(len(non_exist_images.shape)) == 4:
                        non_exist_images = non_exist_images[0,:, :, :]
                        
                    image_letter = subtractImages(non_exist_images,image)
                    #image_letter = cv2.resize(image_letter, (128, 128))
                    X1.append(image_letter)
                    y1.append(1)

                    for i in range(1,3):
                        
                        image = copy.deepcopy(Image.open(data +'/'+ folder_name + '/' + filename))
                        r0,r1,new_img = random_att(additions0,additions1,image)
                        exist_images = findImageLetterB(fin_image_name,r0,r1)

                        if exist_images:
                            image_letter = subtractImages(exist_images[0],new_img)
                            #image_letter = cv2.resize(image_letter, (128, 128))
                            X1.append(image_letter)
                            y1.append(1)
                        
                        else :
                            non_exist_images = findImageNotLetter(fin_image_name)
                    
                            #print(filename)
                            non_exist_images = np.asarray(non_exist_images)
                            
                            if(len(non_exist_images.shape)) == 4:
                                non_exist_images = non_exist_images[0,:, :, :]
                                
                            image_letter = subtractImages(non_exist_images,new_img)
                            #image_letter = cv2.resize(image_letter, (128, 128))
                            X1.append(image_letter)
                            y1.append(1)
                            
                    c_i += 1 

            else :
                
                #image = cv2.resize(image, (128, 128))
                X1.append(image)
                y1.append(0)
                
    X1 = np.asarray(X1)
    y1 = np.asarray(y1)
    #X2 = np.asarray(X2)
    #y2 = np.asarray(y2)

    return X1,y1

X_train, y_train = load_subdata(r'dataset/training/training')



100%|██████████| 1425/1425 [06:36<00:00,  3.59it/s]


### **Train Test Split**

In [9]:
X_train1, X_test, y_train1, y_test = train_test_split(X_train, y_train, test_size=0.10, random_state = 42)
X_train_fin, X_val, y_train_fin, y_val = train_test_split(X_train1, y_train1, test_size=0.10, random_state = 42)

#For training data
y_trainHot = np.uint8(to_categorical(y_train_fin, num_classes = 2))

#For test data
y_testHot = np.uint8(to_categorical(y_test, num_classes = 2))

#For validation data
y_val = np.uint8(to_categorical(y_val,num_classes=2))

In [10]:
s=y_train.tolist()

In [11]:
s.count(0)

4450

### **Importing Packages for Model Training**

In [12]:
from keras.models import Model
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Input
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import GlobalMaxPooling2D
from keras.layers import GlobalAveragePooling2D
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras import backend as K
from keras.applications.imagenet_utils import decode_predictions
from keras.applications.imagenet_utils import preprocess_input
from tensorflow.keras import optimizers


### Customized VGG model

In [14]:
def VGGupdated(input_tensor=None, classes=2):
    img_rows, img_cols = 64,64  # by default size is 224,224
    img_channels = 3

    img_dim = (img_rows, img_cols, img_channels)

    img_input = Input(shape=img_dim)

    # Block 1
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1')(img_input)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

    # Block 2
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

    # Block 3
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

    # Block 4
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

    # Block 5
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv1')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv2')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(x)

    # Classification block
    x = Flatten(name='flatten')(x)
    x = Dense(4096, activation='relu', name='fc1')(x)
    x = Dense(4096, activation='relu', name='fc2')(x)
    x = Dense(classes, activation='softmax', name='predictions')(x)

    # Create model.

    model = Model(inputs=img_input, outputs=x, name='VGGdemo')

    return model

model = VGGupdated(classes = 2)

model.compile(optimizer=optimizers.Adamax(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

**Fitting the model**

In [15]:
print(X_train_fin.shape)
print(y_trainHot.shape)

model.fit(X_train_fin,y_trainHot, epochs=10,validation_data = (X_val,y_val), steps_per_epoch=2000)

(7066, 64, 64, 3)
(7066, 2)
Epoch 1/10
2000/2000 [==============================] - 168s 68ms/step - loss: 0.0773 - accuracy: 0.9739 - val_loss: 0.0040 - val_accuracy: 0.9987
Epoch 2/10
2000/2000 [==============================] - 134s 67ms/step - loss: 0.0055 - accuracy: 0.9987 - val_loss: 0.0136 - val_accuracy: 0.9987
Epoch 3/10
2000/2000 [==============================] - 133s 67ms/step - loss: 0.0027 - accuracy: 0.9992 - val_loss: 8.5036e-04 - val_accuracy: 1.0000
Epoch 4/10
2000/2000 [==============================] - 132s 66ms/step - loss: 1.2037e-05 - accuracy: 1.0000 - val_loss: 6.9071e-05 - val_accuracy: 1.0000
Epoch 5/10
2000/2000 [==============================] - 133s 66ms/step - loss: 5.6337e-07 - accuracy: 1.0000 - val_loss: 1.6628e-05 - val_accuracy: 1.0000
Epoch 6/10
2000/2000 [==============================] - 133s 67ms/step - loss: 3.6278e-08 - accuracy: 1.0000 - val_loss: 8.1279e-06 - val_accuracy: 1.0000
Epoch 7/10
2000/2000 [==============================] - 133s 6

Saving the **Weight**

In [16]:
from keras.models import model_from_json

model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

model.save_weights("model.h5")
print("Saved model to disk")



Saved model to disk


 Loading the **Weight**

In [17]:
from keras.models import model_from_json
with open("model.json") as json_file:
    model = model_from_json(json_file.read())
    model.load_weights("model.h5")

# **Tranfer learning** Starts......

### Loading the **DATASET** (for transfer learning)

1.   Background with letters(1)
2.   Background without letter (0)



In [18]:
#To load data normally
def load_data(data):
    X1 = []
    y1 = []

    for folder_name in os.listdir(data):

        if not folder_name.startswith('.'):
            
            if folder_name in ['background'] :
                label = 0;
            elif folder_name in ['hi'] :
                label = 1;

        for filename in tqdm(os.listdir(data + '/' + folder_name)):
            
            image = copy.deepcopy(np.array(Image.open(data +'/'+ folder_name + '/' + filename),dtype=np.uint8))
    
            image = np.asarray(image);
            
            image_data_as_arr = np.asarray(image)

            X1.append(image_data_as_arr)
            y1.append(label)
                

    X1 = np.asarray(X1)
    y1 = np.asarray(y1)

    return X1,y1

imgs, img_labels = load_data(r'dataset/training/training')

X_train_tf, X_test_tf, y_train_tf, y_test_tf = train_test_split(imgs, img_labels, test_size=0.10, shuffle=True)
X_train_fin_tf, X_val_tf, y_train_fin_tf, y_val_tf = train_test_split(X_train_tf, y_train_tf, test_size=0.10, shuffle=True)




100%|██████████| 1425/1425 [00:02<00:00, 657.10it/s]


In [19]:
len(X_train)

8725

In [20]:
from keras.utils.np_utils import to_categorical
#For training data
y_trainHot_tf = np.uint8(to_categorical(y_train_fin_tf, num_classes = 2))

#For test data
y_testHot_tf = np.uint8(to_categorical(y_test_tf, num_classes = 2))

#For validation data
y_valHot_tf = np.uint8(to_categorical(y_val_tf,num_classes=2))

In [21]:
len(y_testHot_tf)

588

### **Freezing the Layers**

In [22]:
model.summary()

Model: "VGGdemo"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 64, 64, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 64, 64, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 64, 64, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 32, 32, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 32, 32, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 32, 32, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 16, 16, 128)       0   

In [23]:
from keras import Model
n=6
for i in range(1,n):
    model.layers[i].trainable = False
for i in range(n,18):
    model.layers[i].trainable = True

# for i in range(18,22):
#     model.layers[i].trainable = True

l1 = model.layers[18].output
l1= Flatten()(l1)
l1 = Dense(4096, activation='relu')(l1)
l1 = Dense(4096, activation='relu')(l1)
l1 = Dense(2, activation='sigmoid')(l1)

new_model = Model(inputs=model.input,outputs=l1)


new_model.compile(optimizers.Adamax(learning_rate=0.0001),
                  loss='binary_crossentropy',
                  metrics=['binary_accuracy'])

history = new_model.fit(X_train_fin_tf,y_trainHot_tf,shuffle = True,epochs = 60,validation_data= (X_val_tf,y_valHot_tf),steps_per_epoch=2000)

Epoch 1/60
2000/2000 [==============================] - 114s 56ms/step - loss: 0.2772 - binary_accuracy: 0.9010 - val_loss: 0.1521 - val_binary_accuracy: 0.9518
Epoch 2/60
2000/2000 [==============================] - 112s 56ms/step - loss: 0.1477 - binary_accuracy: 0.9463 - val_loss: 0.1534 - val_binary_accuracy: 0.9556
Epoch 3/60
2000/2000 [==============================] - 112s 56ms/step - loss: 0.0956 - binary_accuracy: 0.9683 - val_loss: 0.0886 - val_binary_accuracy: 0.9707
Epoch 4/60
2000/2000 [==============================] - 111s 56ms/step - loss: 0.0578 - binary_accuracy: 0.9803 - val_loss: 0.0728 - val_binary_accuracy: 0.9792
Epoch 5/60
2000/2000 [==============================] - 112s 56ms/step - loss: 0.0356 - binary_accuracy: 0.9873 - val_loss: 0.0646 - val_binary_accuracy: 0.9792
Epoch 6/60
2000/2000 [==============================] - 111s 56ms/step - loss: 0.0235 - binary_accuracy: 0.9927 - val_loss: 0.0673 - val_binary_accuracy: 0.9830
Epoch 7/60
2000/2000 [============

Saving the Model

In [24]:
new_model.save("modelTL.h5")

# Validating with test data

In [36]:
def validate_set(data):

    X_valid = []

    for i in range(1,99):
        image = Image.open(data +"/" +str(i)+".jpg")
        
        image = np.array(image)

        image_data_as_arr = np.asarray(image)
        
        X_valid.append(image_data_as_arr)

    X_valid = np.asarray(X_valid)
    
    X_valid = tf.expand_dims(X_valid, axis=-1)
    
    return X_valid


In [37]:
X_val = validate_set(r'dataset/test/test')

y_val = [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0,
  1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 
  0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0];

y_val = np.uint8(to_categorical(y_val, num_classes = 2));

y_pred = new_model.predict(X_val)
Y_pred_classes = np.argmax(y_pred,axis=1)
print(Y_pred_classes)

print((X_val).shape)

[1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0
 0 0 1 0 1 0 0 0 1 0 1 0 1 1 1 1 1 1 0 0 0 0 0 1 1 1 0 0 0 1 0 1 1 1 1 1 0
 0 0 0 0 0 0 1 1 1 1 0 0 0 0 0 1 1 1 1 1 0 0 0 0]
(98, 64, 64, 3, 1)


In [38]:
score = new_model.evaluate(X_val,y_val, verbose=1)
print('\nKeras CNN binary accuracy:', score[1],'\n')


4/4 [==============================] - 0s 37ms/step - loss: 0.4538 - binary_accuracy: 0.9796

Keras CNN binary accuracy: 0.9795918464660645 



In [39]:
from tensorflow import keras
mod = keras.models.load_model("modelTL.h5")

In [40]:
X_val = validate_set(r'dataset/test/test')

y_val = [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0,
  1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 
  0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0];

y_val = np.uint8(to_categorical(y_val, num_classes = 2));

y_pred = mod.predict(X_val)
Y_pred_classes = np.argmax(y_pred,axis=1)
print(Y_pred_classes)

print((X_val).shape)

score = mod.evaluate(X_val,y_val, verbose=1)
print('\nKeras CNN binary accuracy:', score[1],'\n')

[1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0
 0 0 1 0 1 0 0 0 1 0 1 0 1 1 1 1 1 1 0 0 0 0 0 1 1 1 0 0 0 1 0 1 1 1 1 1 0
 0 0 0 0 0 0 1 1 1 1 0 0 0 0 0 1 1 1 1 1 0 0 0 0]
(98, 64, 64, 3, 1)
4/4 [==============================] - 0s 38ms/step - loss: 0.4538 - binary_accuracy: 0.9796

Keras CNN binary accuracy: 0.9795918464660645 



Validating test data test2_0

In [31]:
def validate_set(data):

    X_valid = []

    for i in range(1,5):
        image = Image.open(data +"/" +str(i)+".jpeg")
        
        image = np.array(image)

        image_data_as_arr = np.asarray(image)
        
        X_valid.append(image_data_as_arr)

    X_valid = np.asarray(X_valid)
    
    X_valid = tf.expand_dims(X_valid, axis=-1)
    
    return X_valid


In [32]:
X_val = validate_set(r'test_2_0')

In [33]:
y_pred = mod.predict(X_val)
Y_pred_classes = np.argmax(y_pred,axis=1)
print(Y_pred_classes)

[1 1 1 1]


In [ ]:
import PIL

files = os.listdir(r'dataset/test/test')

rows = 10

for i in range(1,99):
    
    plt.figure(figsize=(64, 64))
    img = PIL.Image.open(r'dataset/test/test/' + str(i)+".jpg")
    plt.subplot(rows,10,i)
    plt.title(Y_pred_classes[i-1])
    plt.axis('off')
    plt.imshow(img)